In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import skimage
import scipy
import os
import h5py
from skimage.exposure import histogram

from calibrations import itof_calibration

file = r"J:\ctgroup\Edward\DATA\VMI\20250408\po_800mW_lin.cv4"

with h5py.File(file, 'r') as f:
    t_tof = f['t_tof'][:]
    tof_corr = f['tof_corr'][:]

t_tof, tof_corr = t_tof[(t_tof < 2e4) & (t_tof > 0)], tof_corr[(t_tof < 2e4) & (t_tof > 0)]

mq = itof_calibration(t_tof)
px.histogram(x=mq).show()

In [ ]:
df = pd.DataFrame({'t_tof': t_tof, 'tof_corr': tof_corr, 'mq': mq})
groups = df.groupby('tof_corr')
pairs = []
for _, group in groups:
    if len(group) == 2:
        pairs.append(group['mq'].to_numpy())
        pairs.append(group['mq'].to_numpy()[::-1])


In [ ]:
pairs = np.array(pairs)
n = 300
h2d = np.histogram2d(pairs[:, 0], pairs[:, 1], bins=n, range=[(0, 65)] * 2)[0]
h1d1, e = np.histogram(pairs[:, 0], bins=n, range=(0, 65))
h1d2, e = np.histogram(pairs[:, 1], bins=n, range=(0, 65))
h1df, e = np.histogram(mq, bins=n, range=(0, 65), density=True)
h2dp = h1df[:, np.newaxis] * h1df[np.newaxis, :]
xind, yind = np.meshgrid(np.arange(n), np.arange(n))
norm_idx = np.argmin(np.abs(e - 55.3))
h1d = h1d1 / h1d1[norm_idx]

pt1 = (28 * n / 128, 37 * n / 128)
pt2 = (112 * n / 128, 127 * n / 128)

gap = np.where(
        (yind - ((pt2[0] - pt1[0]) / (pt2[1] - pt1[1]) * (xind - pt1[1]) + pt1[0]) > -14)
        & (xind - ((pt2[0] - pt1[0]) / (pt2[1] - pt1[1]) * (yind - pt1[1]) + pt1[0]) > -14),
        0,
        1
)

gate = (35, 50)
itof_gated = pairs[(pairs[:, 0] > gate[0]) & (pairs[:, 0] < gate[1])][:, 1]
itof_gated_hist = np.histogram(itof_gated, bins=n, density=True, range=(0, 65))[0]
itof_gated_hist /= itof_gated_hist[norm_idx]

gate2 = (20, 35)
itof_gated2 = pairs[(pairs[:, 0] > gate2[0]) & (pairs[:, 0] < gate2[1])][:, 1]
itof_gated_hist2 = np.histogram(itof_gated2, bins=n, density=True, range=(0, 65))[0]
itof_gated_hist2 /= itof_gated_hist2[norm_idx]

gate3 = (12, 15)
itof_gated3 = pairs[(pairs[:, 0] > gate3[0]) & (pairs[:, 0] < gate3[1])][:, 1]
itof_gated_hist3 = np.histogram(itof_gated3, bins=n, density=True, range=(0, 65))[0]
itof_gated_hist3 /= itof_gated_hist3[norm_idx]

h2d *= gap
h2d = scipy.ndimage.gaussian_filter(h2d, sigma=5)
h2d /= np.sum(h2d) / n ** 2

(
    px.line(x=e[:-1], y=h1d)
    .add_scatter(x=e[:-1], y=itof_gated_hist, name="Gated on C2H3O+")
    .add_scatter(x=e[:-1], y=itof_gated_hist2, name="Gated on C2H4+")
    .add_scatter(x=e[:-1], y=itof_gated_hist3, name="Gated on CH3+")
    .update_xaxes(title="m/q")
    .update_yaxes(title="Count (Relative to parent ion)")
    .show()
)

(px.imshow(
        h2d,
        aspect='equal',
        origin='lower',
)
 .show())

# px.imshow(h2dp, aspect='auto', origin='lower').show()
h2dp *= gap
h2dp = scipy.ndimage.gaussian_filter(h2dp, sigma=5)
h2dp /= np.sum(h2dp) / n ** 2
px.imshow(h2dp, aspect='equal', origin='lower').show()

px.imshow(h2d - h2dp,
          aspect='equal',
          origin='lower',
          color_continuous_scale="balance",
          color_continuous_midpoint=0
          ).show()

px.imshow((h2d - h2dp) * 2 / (h2dp + h2d) * 1 / (h2dp / np.max(h2dp) > 0.05),
          aspect='equal',
          origin='lower',
          color_continuous_scale="balance",
          color_continuous_midpoint=0
          ).show()